In [1]:
%config InlineBackend.rc = {"figure.dpi": 72, 'figure.figsize': (6.0, 4.0)}
%matplotlib ipympl

import ase 
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import HTML

import abtem

abtem.config.set({"local_diagnostics.progress_bar": True})
abtem.config.set({"device": "cpu"});

(examples:interactions)=
# Interactive visualizations

We can make simple interactive visualizations using iPython.

```{warning}
Interactive visualizations require `ipympl` to be installed! If you get a "Widgets loading..." message and the visualization does not show, you may need to follow [these instructions](https://ipywidgets.readthedocs.io/en/latest/user_install.html#installing-in-jupyterlab-3-x).
```

## STEM

We create a model for MoS$_2$ with a scanned probe, where the semiangle cutoff, defocus, and spherical aberration are ensembles. In addition, we apply Poisson noise to the measurement, giving yet another slider to visualize. Finally, by using the default `FlexibleAnnularDetector` detector, the inner and outer radial scattering angles become freely controllable.

In [9]:
atoms = ase.build.mx2(vacuum=3)
atoms = abtem.orthogonalize_cell(atoms) * (3, 2, 1)

probe = abtem.Probe(
    semiangle_cutoff=np.linspace(20, 30, 3),
    defocus=np.linspace(0, 50, 3),
    Cs=np.linspace(0, 5e5, 3),
    astigmatism_angle=np.pi / 4,
    sampling=0.05,
    energy=80e3,
)

potential = abtem.Potential(atoms, projection="infinite", slice_thickness=2)

scan = abtem.GridScan(
    start=(0, 0),
    end=(1 / 3, 1 / 2),
    fractional=True,
    potential=potential,
)

doses = np.geomspace(1e1, 1e7, 7)

measurements = (
    probe.scan(potential, scan=scan)
    .to_image_ensemble()
    .gaussian_filter(0.3)
    .interpolate(0.1)
    .tile((3, 2))
    .compute()
    .poisson_noise(dose_per_area=doses)
    .compute()
)

[########################################] | 100% Completed | 7.85 ss


The `show` method now creates a widget where each ensemble can be separately controlled.

Note that to see atomic contrast, you need to increase the dose and set the radial scattering angles to reasonable values, e.g. 40 to 274 mrad. Selecting Autoscale will automatically adapt the contrast.

In [12]:
animation = measurements.show(interact=True, cbar=True)